In [1]:
import os
work_dir = "/content/tensorflow-tutorial/"
if os.getcwd() != work_dir:
    !git clone https://github.com/sewerynzalupka/tensorflow-tutorial.git
os.chdir(work_dir)

# !pip install -r requirements.txt

Cloning into 'tensorflow-tutorial'...
remote: Enumerating objects: 503, done.
remote: Counting objects: 100% (503/503), done.
remote: Compressing objects: 100% (273/273), done.
remote: Total 503 (delta 242), reused 483 (delta 225), pack-reused 0
Receiving objects: 100% (503/503), 62.71 MiB | 29.00 MiB/s, done.
Resolving deltas: 100% (242/242), done.


# Imports

In [2]:
%matplotlib inline
import PIL
import os
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.13.1


# Dataset - SigComp2011

Download and extract training and test sets (or use cached files)

In [3]:
import signatures

signatures.maybe_download_and_extract()

- Download progress: 100.0%
Download finished. Extracting files.
- Extracting progress: 100.0%Done.
- Download progress: 100.0%
Download finished. Extracting files.
- Extracting progress: 100.0%Done.


In [53]:
!ls 

01_Simple_Linear_Model.ipynb	       cifar10.py
02_Convolutional_Neural_Network.ipynb  coco.py
03B_Layers_API.ipynb		       convert.py
03C_Keras_API.ipynb		       data
03_PrettyTensor.ipynb		       dataset.py
04_Save_Restore.ipynb		       download.py
05_Ensemble_Learning.ipynb	       europarl.py
06_CIFAR-10.ipynb		       forks.md
07_Inception_Model.ipynb	       images
08_Transfer_Learning.ipynb	       imdb.py
09_Video_Data.ipynb		       inception5h.py
10_Fine-Tuning.ipynb		       inception.py
11_Adversarial_Examples.ipynb	       knifey.py
12_Adversarial_Noise_MNIST.ipynb       LICENSE
13B_Visual_Analysis_MNIST.ipynb        mnist.py
13_Visual_Analysis.ipynb	       __pycache__
14_DeepDream.ipynb		       README.md
15_Style_Transfer.ipynb		       reinforcement_learning.py
16_Reinforcement_Learning.ipynb        requirements.txt
17_Estimator_API.ipynb		       signatures_dataset.py
18_TFRecords_Dataset_API.ipynb	       signatures.py
19_Hyper-Parameters.ipynb	       Signature_Verification.ipynb

This dataset has different directory structure than the Keras API requires, so copy the files into separate directories for the training- and test-sets.

In [37]:
from PIL import Image
import pickle
import cv2

def NormalizedImage(img, new_height, new_width):
  cImg = 255*np.ones((new_height,new_width,3))
  width =  np.size(img,1)
  height = np.size(img,0)
  left = int((new_width - width)/2.)
  top = int(((new_height - height)/2.))
  right = int(((width + new_width)/2.))
  bottom = int(((height + new_height)/2.))
  cImg[top:bottom, left:right,:] = img
  return cImg

path = os.listdir('trainingSet/OfflineSignatures/Dutch/TrainingSet/Offline Genuine/')
path.sort(key=str.lower)

print(path)
print(path[0], path[0][:3])

def get_filenames(ext, dir):
  filenames = []
  for root, dirs, files in os.walk(dir):
    for file in files:
      if file.lower().endswith(ext.lower()):
        filenames.append(file)
  return filenames

print(get_filenames('.png', 'trainingSet/OfflineSignatures/Dutch/TrainingSet/Offline Genuine/'))

import re
forgery_filename_pattern = r"(?P<forger>\d{4})(?P<signer>\d{3})_(?P<i>\d{2})"
genuine_filename_pattern = r"(?P<signer>\d{3})_(?P<i>\d{2})"

signers = set()
for filename in get_filenames('.png', 'trainingSet/OfflineSignatures/Dutch/'):
#   forgery_match = re.match(forgery_filename_pattern, filename)
  genuine_match = re.match(genuine_filename_pattern, filename)
#   if forgery_match:
#     signers.add(forgery_match.group('signer'))
  if genuine_match:
    signers.add(genuine_match.group('signer'))
    
print(sorted(list(signers)))

['001_01.PNG', '001_02.PNG', '001_03.PNG', '001_04.PNG', '001_05.PNG', '001_06.PNG', '001_07.PNG', '001_08.PNG', '001_09.PNG', '001_10.PNG', '001_11.PNG', '001_12.PNG', '001_13.PNG', '001_14.PNG', '001_15.PNG', '001_16.PNG', '001_17.PNG', '001_18.PNG', '001_19.PNG', '001_20.PNG', '001_21.PNG', '001_22.PNG', '001_23.PNG', '001_24.PNG', '002_01.PNG', '002_02.PNG', '002_03.PNG', '002_04.PNG', '002_05.PNG', '002_06.PNG', '002_07.PNG', '002_08.PNG', '002_09.PNG', '002_10.PNG', '002_11.PNG', '002_12.PNG', '002_13.PNG', '002_14.PNG', '002_15.PNG', '002_16.PNG', '002_17.PNG', '002_18.PNG', '002_19.PNG', '002_20.PNG', '002_21.PNG', '002_22.PNG', '002_23.PNG', '002_24.PNG', '003_01.PNG', '003_02.PNG', '003_03.PNG', '003_04.PNG', '003_05.PNG', '003_06.PNG', '003_07.PNG', '003_08.PNG', '003_09.PNG', '003_10.PNG', '003_11.PNG', '003_12.PNG', '003_13.PNG', '003_14.PNG', '003_15.PNG', '003_16.PNG', '003_17.PNG', '003_18.PNG', '003_19.PNG', '003_20.PNG', '003_21.PNG', '003_22.PNG', '003_23.PNG', '003_

In [54]:
import numpy as np
import os
import shutil
from cache import cache
import re

forgery_filename_pattern = r"(?P<forger>\d{{4}})(?P<signer>\d{{3}})_(?P<i>\d{{2}})({exts})$"
genuine_filename_pattern = r"(?P<signer>\d{{3}})_(?P<i>\d{{2}})({exts})$"

class SignaturesDataset:
    def __init__(self, train_dir, test_dir, exts=['.png']):
        """
        Create a data-set consisting of the filenames in the given directory
        and sub-dirs that match the given filename-extensions.

        :param train_dir:
            Root-dir for the files in the data-set.
            
        :param test_dir:
            Root-dir for the files in the data-set.

        :param exts:
            String or tuple of strings with valid filename-extensions.
            Not case-sensitive.

        :return:
            Object instance.
        """

        # Extend input directories to the full path.
        train_dir = os.path.abspath(train_dir)
        test_dir = os.path.abspath(test_dir)

        # Input directories.
        self.train_dir = train_dir
        self.test_dir = test_dir

        # Convert all file-extensions to lower-case.
        self.exts = tuple(ext.lower() for ext in exts)

        # Train set
        signatures = {}
        for filename in self._get_filenames(self.train_dir):
            forgery_match = re.search(forgery_filename_pattern.format(exts='|'.join(self.exts)), filename.lower())
            if forgery_match:
                forged_signer = forgery_match.group('signer')
                if forged_signer not in signatures:
                    signatures[forged_signer] = { 'genuine': [], 'forgeries': [] }
                signatures[forged_signer]['forgeries'].append(filename)
                continue
            genuine_match = re.search(genuine_filename_pattern.format(exts='|'.join(self.exts)), filename.lower())
            if genuine_match:
                signer = genuine_match.group('signer')
                if signer not in signatures:
                    signatures[signer] = { 'genuine': [], 'forgeries': [] }
                signatures[signer]['genuine'].append(filename)
        
        authors = list(signatures.keys())
        
        self.signatures_dict = signatures
        
        self.genuine_classes = ['{author}_genuine'.format(author=author) for author in authors]
        self.forged_classes = ['{author}_forged'.format(author=author) for author in authors]
        self.classes = self.genuine_classes + self.forged_classes
        
        self.num_classes = len(self.classes)
                
    def _get_filenames(self, dir):
        """
        Create and return a list of filenames with matching extensions in the given directory.

        :param dir:
            Directory to scan for files. Sub-dirs are not scanned.

        :return:
            List of filenames. Only filenames. Does not include the directory.
        """
        filenames = []
        for root, dirs, files in os.walk(dir):
            for file in files:
                if file.lower().endswith(self.exts):
                    filenames.append(os.path.join(root, file))
        return filenames

test_dir = 'Testdata_SigComp2011/SigComp11-Offlinetestset/Dutch/'
train_dir='trainingSet/OfflineSignatures/Dutch/TrainingSet/'
ds = SignaturesDataset(test_dir=test_dir, train_dir=train_dir)
print(ds.classes)
print(ds.num_classes)
# print(ds.train_dir, ds.test_dir)
# print(list(ds.train_signatures_dict.keys()))
# for author in list(ds.train_signatures_dict.keys()):
#     print(author)
#     print('genuine signatures count: ', len(ds.train_signatures_dict[author]['genuine']))
#     print(ds.train_signatures_dict[author]['genuine'])
#     print('forged signatures count: ', len(ds.train_signatures_dict[author]['forgeries']))
#     print(ds.train_signatures_dict[author]['forgeries'])

def load_cached(cache_path, train_dir, test_dir):
    """
    Wrapper-function for creating a DataSet-object, which will be
    loaded from a cache-file if it already exists, otherwise a new
    object will be created and saved to the cache-file.

    This is useful if you need to ensure the ordering of the
    filenames is consistent every time you load the data-set,
    for example if you use the DataSet-object in combination
    with Transfer Values saved to another cache-file, see e.g.
    Tutorial #09 for an example of this.

    :param cache_path:
        File-path for the cache-file.

    :param in_dir:
        Root-dir for the files in the data-set.
        This is an argument for the DataSet-init function.

    :return:
        The DataSet-object.
    """

    print("Creating dataset from the training files in " + train_dir + " and the testing files in " + test_dir)

    # If the object-instance for SignaturesDataset already
    # exists in the cache-file then reload it, otherwise create
    # an object instance and save it to the cache-file for next time.
    dataset = cache(cache_path=cache_path,
                    fn=SignaturesDataset,
                    train_dir=train_dir,
                    test_dir=test_dir)

    return dataset
  
load_cached(os.path.abspath("signatures.pkl"), train_dir=train_dir, test_dir=test_dir)

['002_genuine', '009_genuine', '014_genuine', '001_genuine', '004_genuine', '003_genuine', '012_genuine', '006_genuine', '016_genuine', '015_genuine', '002_forged', '009_forged', '014_forged', '001_forged', '004_forged', '003_forged', '012_forged', '006_forged', '016_forged', '015_forged']
20
Creating dataset from the training files in trainingSet/OfflineSignatures/Dutch/TrainingSet/ and the testing files in Testdata_SigComp2011/SigComp11-Offlinetestset/Dutch/
- Data saved to cache-file: /content/tensorflow-tutorial/signatures.pkl


In [55]:
!ls

01_Simple_Linear_Model.ipynb	       cifar10.py
02_Convolutional_Neural_Network.ipynb  coco.py
03B_Layers_API.ipynb		       convert.py
03C_Keras_API.ipynb		       data
03_PrettyTensor.ipynb		       dataset.py
04_Save_Restore.ipynb		       download.py
05_Ensemble_Learning.ipynb	       europarl.py
06_CIFAR-10.ipynb		       forks.md
07_Inception_Model.ipynb	       images
08_Transfer_Learning.ipynb	       imdb.py
09_Video_Data.ipynb		       inception5h.py
10_Fine-Tuning.ipynb		       inception.py
11_Adversarial_Examples.ipynb	       knifey.py
12_Adversarial_Noise_MNIST.ipynb       LICENSE
13B_Visual_Analysis_MNIST.ipynb        mnist.py
13_Visual_Analysis.ipynb	       __pycache__
14_DeepDream.ipynb		       README.md
15_Style_Transfer.ipynb		       reinforcement_learning.py
16_Reinforcement_Learning.ipynb        requirements.txt
17_Estimator_API.ipynb		       signatures_dataset.py
18_TFRecords_Dataset_API.ipynb	       signatures.pkl
19_Hyper-Parameters.ipynb	       signatures.py
20_Natural_La